<hr style="border-top: 3px double #8c8b8b;"/>
<br/>
<center><font size=7 color="green">データサイエンスII   第1回 実習</font><br/>
2022.4.12
</center>
<hr style="border-top: 3px double #8c8b8b;"/>


## 最初に次を実行

In [ ]:
import numpy as np

def showgraph(flist,labels,xrange=[0,1],K=50):
    import plotly.graph_objects as go
    import numpy as np
    fig = go.Figure()
    tmin,tmax=xrange
    ts=np.linspace(tmin,tmax,K)
    for i in range(len(flist)):
        fig.add_trace(go.Scatter(x=ts, y=[flist[i](t) for t in ts], name=labels[i]))
    if len(flist)==1:
        fig.update_layout(title=labels[0])
    fig.show() 
def showgraph2(f,xrange=[-1,1],yrange=[-1,1],K=50,opacity=0.5):
    import numpy as np
    import plotly.graph_objects as go
    xmin,xmax=xrange
    ymin,ymax=yrange
    a0=np.linspace(xmin,xmax,K)
    b0=np.linspace(ymin,ymax,K)
    xx,yy=np.meshgrid(a0,b0)
    F=go.Figure()
    F.add_surface(x=xx,y=yy,z=f(xx,yy),opacity=opacity)   
    F.show()
def toukousen(f,xrange=[0,1],yrange=[0,1],T=20,start=0,end=0,size=1):
    import numpy as np
    import plotly.graph_objects as go
    xmin,xmax=xrange
    ymin,ymax=yrange
    a0=np.linspace(xmin,xmax,T)
    b0=np.linspace(ymin,ymax,T)
    x,y=np.meshgrid(a0,b0)
    F=go.Figure()
    F.add_contour(x=a0,y=b0,z=f(x,y),colorscale="oranges",
        contours=dict(
            start=start, #2 等高線の高さの最小値
            end=end,    #3 等高線の高さの最大値
            size=size, #4 描く間隔
            coloring='lines',
            showlabels=True
        ))
    F.update_yaxes(scaleanchor="x", scaleratio=1)
    F.show()
def heikin(D):    
    return sum(D)/len(D)
def bunsan(D):
    if len(D)>0:
        m=heikin(D)
        return sum((D-m)**2)
    else:
        return 0
def mode(D):
    D=sorted(D)
    k=len(D)//2
    if len(D)%2==1:
        return D[k]
    else:
        return (D[k-1]+D[k])/2.
def l1bunsan(D):
    if len(D)>0:
        m=mode(D)
        return sum(abs(D-m))
    else:
        return 0

<hr>

# 「データサイエンス」の典型的問題
+ 問題. $X$の部分集合$D$ (データ集合) と、ラベル$f: D\rightarrow V$ ( $V$は有限集合 )が与えられているとき、$D$以外の要素のラベルを推定せよ.
+ 解法1. 具体的な写像族 $F: X \times \Lambda \rightarrow V$を設定し、$ F(\bullet,\lambda)|_D \approx f$ となる$\lambda\in \Lambda$を、$\Lambda$上の誤差関数を最小化してもとめる.
+ 解法2. $D$をグループに分けて各グループ上ではラベルは同じとなるようにし、一般の$ X$ の要素のラベルを、それに最も近いグループのラベルとして推定する. (グループ数を小さくすることがポイント)

# 1 実習1. 実数データ集合を１実数で近似する問題

## 1.1 実数データを１実数で近似

実数のデータの集まり $D=\{d_0,d_1,\cdots,d_{N-1}\}$ 
を実数$a$で近似する問題を考えよう。これは$D$を代表する典型的な実数$a$を求めることにほかならない。
近似の度合はどのように定量化できるだろうか。

データが一つの場合、つまり$D=\{d\}$の場合は、$|d-a|$ が近似度を示しているが$|d-a|^2$ でも構わない。いずれにしても$0$となるのが最良の近似$d$を与えている。

しかし、$D$が複数あるときは、近似度のとり方は色々ありえる。
  $$gosa_2(x,D)=(x-d_0)^2+(x-d_1)^2+\cdots+(x-d_{N-1})^2$$
や
  $$gosa_1(x,D)=|x-d_0|+|x-d_1|+\cdots+|x-d_{N-1}|$$
が自然である。$gosa_i$を $L_i$誤差関数と呼ぶことにする。


$L_2$誤差関数を最小にする実数を求めてみよう。

## 1.2 L2誤差関数

$D=\{1,2,3,6,7,8,9\}$ の場合を調べる。これを numpy の配列とする。まず、numpy を読み込み、名称を `np`とする。

In [ ]:
import numpy as np

$D$は python のリスト `[1,2,3,6,7,8,9]`で表現できるが、 numpy の`ndarray` というデータ構造にしておくと便利なことが多い。このリストを`ndarray`にするには次のようにする。

In [ ]:
D=np.array([1,2,3,6,7,8,9])

$a$と $D$の元$d$の差の二乗$(a-d)^2$は `(a-d)**2`と書く。これを成分とするリストは `[(a-d)**2 for d in D]` であった. このリストの成分の和が誤差関数となる.

In [ ]:
def gosa2(x,D):
    return sum([(x-d)**2 for d in D]) 

+ 注意. `[(x-d)**2 for d in D]`は簡潔に`(x-D)**2`とかけて、`gosa2`は `sum((x-D)**2)`でも定義できる。しかし、この表現を使うと、`x` がリストのときにエラーが生じるので、上のように定義する.

### 1.2.1 誤差関数のグラフ

関数`gosa2(x,D)`のグラフを描いてみる. 最初に$D$を固定して得られる１変数関数 $t\mapsto gosa2(t,D)$ を定義しておく. これはpython で次のように書く.

In [ ]:
gosa2D=lambda t:gosa2(t,D)

In [ ]:
showgraph([gosa2D],["L2誤差"],[0,10])

### 1.2.2 図の操作

この図は、マウスで操作できる. メニューは、セルを選択したときに右上に現れる. 
+ ズームイン：カーソルが+ ではないとき、メニューから虫眼鏡アイコンをクリック。
   + 左ボタンを押したままマウスを動かして範囲を指定. 
   + 上端からマウスを動かすと$x$の動く範囲を制限できる. 左端からだと$y$の範囲を制限できる.
+ 図の移動：カーソルが 
   + 水平方向に動かすには、下端にマウスを近づけアイコンが左右矢印になったら左ボタンを押したまま動かす。上下に動かすには左端で行う.  
   + 任意方向に動かすには、メニューの上下左右アイコンをクリックしたあと、左ボタンを押したまま動かす.
+ もとに戻すには、右上にポインタを置いてでてくるメニューの家アイコンをクリックする 

### 1.2.3 数式処理で最小化を行う

`gosa2(a)`は aの二次多項式なのでその最小値は微分法で簡単に求められる。
数式処理を行うライブラリ `sympy` を使って計算してみよう。

ライブラリ名を `sy`として読み込む

In [ ]:
import sympy as sy

`a`を変数として扱うには以下のようにする。

In [ ]:
from sympy.abc import a

関数 gosa2 の式を `gs2`とする。

In [ ]:
gosa2D=gosa2(a,D)
gosa2D

式を整理する。

In [ ]:
gosa2D.expand()

二次式となったので、その最小値を求めるために微分する。

In [ ]:
gosa2D.diff(a)

これを解くために solve を用いる.

In [ ]:
sol=sy.solve(gosa2D.diff(a),a)

In [ ]:
a=sol[0]
a.n(10)

In [ ]:
gosa2(a,D).n(10)

### 1.2.4 命題. $a$は$D$の平均値. 誤差は分散.
+ 証明  $$\frac{\partial{gosa2(a)}}{\partial{a}}=
  \sum_{d\in D}2(a-d)=2(|D|a-\sum_{d\in D} d).$$
  よって、$\frac{\partial{gosa2(a)}}{\partial{a}}=0$ を$a$について解くと
  $$ a=\frac{\sum D}{|D|},$$
 となり、$a$が$D$の平均値であることがわかり、
$a$における$L_2$誤差
  $$\sum_{d\in D}(d-a)^2$$
は分散であることもわかる.
  



### 1.2.5 平均値と分散との比較

+ `a.n(10)` は、有理数`a`を10桁まで小数展開する.

In [ ]:
a.n(10),heikin(D)

In [ ]:
gosa2(a,D).n(10),bunsan(D)

## 1.3 (参考) L1誤差関数

$L_1$誤差については、どの数が$D$を最もよく近似するであろうか。

In [ ]:
def gosa1(x,D):
    return sum([abs(x-d) for d in D])

### 1.3.1 誤差関数のグラフ

青のグラフが $L_1$誤差関数である.

In [ ]:
gosa1D=lambda t:gosa1(t,D)
showgraph([gosa1D,gosa2D],["L1誤差","L2誤差"],[0,10],K=1000)

+ 赤が$L_2$誤差の曲線、青が$L_1$誤差のグラフである。右欄にある赤の線分をクリックすると青のグラフだけになる。これが折れ線であることがわかり、$6$で最小となっている。これは $L_2$近似での最良近似 $5.14285\dots$よりかなり大きい。

+ gosa1は微分可能でないので数式処理では、2.2.3のように微分して最小値は求められない。そこで数値微分を使って調べる。数値微分は、この講義の後半ではライブラリ`pytorch`を利用するが、当面は素朴に扱う。


### 1.3.2 数値微分

$\varepsilon$を小さくしておけば
$$f'(x)=\lim_{h\rightarrow 0}\frac{f(x+h)-f(x)}{h}\approx \frac{f(x+\varepsilon)-f(x)}{\varepsilon}$$

右辺を f,x,epsilon の関数として定義する.

In [ ]:
def ndiff(f,x,epsilon=0.01):
    return ((f(x+epsilon)-f(x))/epsilon)

In [ ]:
Dgosa1D=lambda x:ndiff(gosa1D,x)
showgraph([gosa1D,Dgosa1D],["gosa1","gosa1の微分"],[0,10])

青をクリックして微分のグラフだけ表示してみると、$x<6$で微分が負、$x>6$で正であることがわかり、$x=6$が最小値なことがわかる。これは$D$の中央値である.
+ 定義. $D=\{d_1\leq d_2\leq \cdots\leq d_N\}$の中央値は
    + $N=2k+1$のとき$d_k$
    + $N=2k$のとき、$(d_k+d_{k+1})/2$ 


# 2 実習2 実数データを２実数で近似する問題

## 2.1 誤差関数

実数の集合$D$ を最もよく近似する２実数$a,b$を求めてみよう。集合$A=\{a,b\}$への実数$x$の隔たり$d(A,x)$を、$x$に最も近い$A$の要素との隔たりとして定義することにする。２実数$x,a$の隔たりとして$|x-a|$をとると,
$$d_1(A,x)=\min_{a\in A}|x-a|$$
となる。平均値の場合と同様に、$x,a$の隔たりを$(x-a)^2$をとれば
$$d_2(A,x)=\min_{a\in A}(x-a)^2$$ となる。
$x$が$D$を動くとき、$x$と$A$の隔たりを足し合わせると$A$が$D$をどの程度近似しているかを表す指標となる。すなわち$L_2$誤差の一般化として
$$ gosa2_2(A,D)=\sum_{d\in D}\min_{a\in A}(d-a)^2$$
$L_1$誤差の一般化として
$$gosa2_1(A,D)=\sum_{d\in D}\min_{a\in A}|d-a|$$
と定めることができる。


## 2.2 L2誤差に関する近似

１点近似を行った$D=\{1,2,3,6,7,8,9\}$ を二点近似してみよう

In [ ]:
D=np.array([1,2,3,6,7,8,9])

### 2.2.1 誤差関数の定義

In [ ]:
def gosa22(a,b,D):
    return sum(min((a-d)**2,(b-d)**2) for d in D)

次の処置は、python の関数を、ndarray に適用できるようにするもの.

In [ ]:
gosa22D=lambda a,b: gosa22(a,b,D)
gs22=np.frompyfunc(gosa22D,2,1)

### 2.2.2 誤差関数のグラフ

In [ ]:
showgraph2(gs22,xrange=[0,10],yrange=[0,10])

#### 2.2.2.1 図をマウスで拡大・縮小・回転することができる。

### 2.2.3 関数の等高線表示

In [ ]:
toukousen(gs22,[0,10],[0,10],start=0,end=10,size=0.1,T=100)

密なところをズームインすると$2,7.5$が最もよく近似していると推測される。この場合のエラーは7となる。$2$は$1,2,3$の平均であり、$7.5$は$6,7,8,9$の平均となっている。

In [ ]:
gosa22(2,7.5,D)

+ 今回も微分可能ではないので数式処理では解けない。

## 2.3 近似実数 2,7.5によるグループ分け

+ a=2,b=7.5 とおく
+ $Da=\{x\in D | |x-a|<|x-b|\}$とおくと、これは$a$の方に近いデータの集合. その補集合を $Db$とする.

In [ ]:
a,b=2,7.5
Da=[u for u in D if abs(u-a)<abs(u-b)]
Db=[u for u in D if abs(u-a)>=abs(u-b)]
Da,Db

+ $2$は$Da$の平均値、$7.5$は  $Db$の平均値となっている. 誤差は$Da$の分散と$Db$の分散の和となっている。

In [ ]:
heikin(Da),heikin(Db),bunsan(Da)+bunsan(Db)

## 2.4 (参考)解法.  グループ分けから、$a,b$をもとめる

+ 方針.  
    + 実数$t$より小さいものを$D1(t)$ $t$以上を$D2(t)$とグループ分けし$a$を$D1(t)$の平均、$b$を$D2(t)$の平均とすると
       `gosa22(a,b,D)=bunsan(D1(t))+bunsan(D2(t))`
    + そこで、$err(t)=bunsan(D1(t))+bunsan(D2(t))$の最小値を求める.
       

In [ ]:
def gosaA2(t,D):
    return bunsan([d for d in D if d<t])+bunsan([d for d in D if d>=t])


In [ ]:
gosaA2D=lambda t:gosaA2(t,D)
showgraph([gosaA2D],["各グループの分散の和"],[0,10])

これより$3<t<6$のときに最小となることがわかる. そのとき$D1(t)=[1,2,3]$,$D2(t)=[6,7,8,9]$で $a=2,b=7.5$

## 2.5 参考. $L_1$誤差についての２点近似

+ 参考3.4 と同様に行う.
+ 平均を中央値、分散を$L_1$分散にする. ただし、$L_1$分散は、中央値とデータの差の絶対値の和である。


In [ ]:
D=np.array([1,2,3,6,7,8,9])

In [ ]:
mode(D),l1bunsan(D)

+ 方針.  
    + 実数$t$より小さいものを$D1(t)$とし、$t$以上を$D2(t)$とグループ分けし$a$を$D1(t)$の中央値、$b$を$D2(t)$の中央値とすると
       `gosa22(a,b,D)=l1bunsan(D1(t))+l1bunsan(D2(t))`
    + そこで、$err1(t)=l1bunsan(D1(t))+l1bunsan(D2(t))$の最小値を求める.
       

In [ ]:
def gosaA1(t,D):
    return l1bunsan([d for d in D if d<t])+l1bunsan([d for d in D if d>=t])

In [ ]:
gosaA1D=lambda t:gosaA1(t,D)
showgraph([gosaA1D],["分割された各グループのL1分散の和"],[0,10])


+ これより、区間[3,6]にある点、たとえば、4 をとり、$4$未満と$4$以上に分離すれば良いことがわかる.この場合は$L_2$誤差の場合と同じである。

In [ ]:
showgraph([gosaA2D,gosaA1D],["グループの分散和","グループのL1分散和"],[0,15])

## 2.6 比較. 一つの要素が孤立している場合

+ $D$の最後の要素$9$を$16$とした場合はどうか?

In [ ]:
D[-1]=16
D

In [ ]:
errD=lambda u:err(u,D)
err1D=lambda u:err1(u,D)
showgraph([errD,err1D],["L2誤差","L1誤差"],[0,15])

+ $L_2$誤差では、[1,2,3,6,7,8]と[16]に分離するときが最小
+ $L_1$誤差では、前と同様に [1,2,3] と [6,7,8,16] に分離するときが最小となる

In [ ]:
D[-1]=9
D

In [ ]:
showgraph([errD,err1D],["L2誤差","L1誤差"],[0,15])